Imports

Import the required libraries.

    #Kaggle's reader needed these.
    import tensorflow as tf
    import re
    from typing import Dict, List, Optional, Text, Tuple
    import matplotlib.pyplot as plt


    from matplotlib import colors
    from scipy.ndimage import generic_filter
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import ConfusionMatrixDisplay
    from sklearn.metrics import r2_score
    import numpy as np
    import random
    from sklearn.preprocessing import StandardScaler
    import seaborn as sns

    from tensorflow import keras
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.metrics import categorical_crossentropy
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    from sklearn.metrics import confusion_matrix
    from keras.layers import Input, Lambda, Dense, Flatten
    from keras.models import Model
    import tensorflow.keras.applications.resnet50
    from tensorflow.keras.applications.resnet50 import ResNet50
    from tensorflow.keras.applications.resnet50 import preprocess_input
    from keras.preprocessing import image
    from keras.preprocessing.image import ImageDataGenerator
    from keras.models import Sequential
    import numpy as np
    from glob import glob
    import matplotlib.pyplot as plt
    import cv2
    import imghdr
    import tensorflow as tf
    import os
    import numpy as np
    from sklearn import datasets
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report, confusion_matrix
    import itertools

Load the dataset

\*\*\*\*Acquired code (denoted by brackets \[\])\*\*\*\* \[

    """Constants for the data reader."""

    INPUT_FEATURES = ['elevation', 'th', 'vs',  'tmmn', 'tmmx', 'sph', 
                      'pr', 'pdsi', 'NDVI', 'population', 'erc', 'PrevFireMask']

    OUTPUT_FEATURES = ['FireMask', ]

    # Data statistics 
    # For each variable, the statistics are ordered in the form:
    # (min_clip, max_clip, mean, standard deviation)
    DATA_STATS = {
        # Elevation in m.
        # 0.1 percentile, 99.9 percentile
        'elevation': (0.0, 3141.0, 657.3003, 649.0147),
        
        # Drought Index (Palmer Drought Severity Index)
        # 0.1 percentile, 99.9 percentile
        'pdsi': (-6.12974870967865, 7.876040384292651, -0.0052714925, 2.6823447),
        
        #Vegetation index (times 10,000 maybe, since it's supposed to be b/w -1 and 1?)
        'NDVI': (-9821.0, 9996.0, 5157.625, 2466.6677),  # min, max
       
        # Precipitation in mm.
        # Negative values do not make sense, so min is set to 0.
        # 0., 99.9 percentile
        'pr': (0.0, 44.53038024902344, 1.7398051, 4.482833),
       
        # Specific humidity.
        # Negative values do not make sense, so min is set to 0.
        # The range of specific humidity is up to 100% so max is 1.
        'sph': (0., 1., 0.0071658953, 0.0042835088),
        
        # Wind direction in degrees clockwise from north.
        # Thus min set to 0 and max set to 360.
        'th': (0., 360.0, 190.32976, 72.59854),
        
        # Min/max temperature in Kelvin.
        
        #Min temp
        # -20 degree C, 99.9 percentile
        'tmmn': (253.15, 298.94891357421875, 281.08768, 8.982386),
        
        #Max temp
        # -20 degree C, 99.9 percentile
        'tmmx': (253.15, 315.09228515625, 295.17383, 9.815496),
        
        # Wind speed in m/s.
        # Negative values do not make sense, given there is a wind direction.
        # 0., 99.9 percentile
        'vs': (0.0, 10.024310074806237, 3.8500874, 1.4109988),
        
        # NFDRS fire danger index energy release component expressed in BTU's per
        # square foot.
        # Negative values do not make sense. Thus min set to zero.
        # 0., 99.9 percentile
        'erc': (0.0, 106.24891662597656, 37.326267, 20.846027),
        
        # Population density
        # min, 99.9 percentile
        'population': (0., 2534.06298828125, 25.531384, 154.72331),
        
        # We don't want to normalize the FireMasks.
        # 1 indicates fire, 0 no fire, -1 unlabeled data
        'PrevFireMask': (-1., 1., 0., 1.),
        'FireMask': (-1., 1., 0., 1.)
    }

The following cell defines cropping functions for extracting regions of
the desired size from the input data.

    """Library of common functions used in deep learning neural networks.
    """
    #YOU PROBABLY WILL NOT USE THESE.

    def random_crop_input_and_output_images(
        input_img: tf.Tensor,
        output_img: tf.Tensor,
        sample_size: int,
        num_in_channels: int,
        num_out_channels: int,
    ) -> Tuple[tf.Tensor, tf.Tensor]:
      """Randomly axis-align crop input and output image tensors.

      Args:
        input_img: tensor with dimensions HWC.
        output_img: tensor with dimensions HWC.
        sample_size: side length (square) to crop to.
        num_in_channels: number of channels in input_img.
        num_out_channels: number of channels in output_img.
      Returns:
        input_img: tensor with dimensions HWC.
        output_img: tensor with dimensions HWC.
      """
      combined = tf.concat([input_img, output_img], axis=2)
      combined = tf.image.random_crop(
          combined,
          [sample_size, sample_size, num_in_channels + num_out_channels])
      input_img = combined[:, :, 0:num_in_channels]
      output_img = combined[:, :, -num_out_channels:]
      return input_img, output_img


    def center_crop_input_and_output_images(
        input_img: tf.Tensor,
        output_img: tf.Tensor,
        sample_size: int,
    ) -> Tuple[tf.Tensor, tf.Tensor]:
      """Center crops input and output image tensors.

      Args:
        input_img: tensor with dimensions HWC.
        output_img: tensor with dimensions HWC.
        sample_size: side length (square) to crop to.
      Returns:
        input_img: tensor with dimensions HWC.
        output_img: tensor with dimensions HWC.
      """
      central_fraction = sample_size / input_img.shape[0]
      input_img = tf.image.central_crop(input_img, central_fraction)
      output_img = tf.image.central_crop(output_img, central_fraction)
      return input_img, output_img

The following cell provides code for parsing the contents of the
TensorFlow Record files. In addition to loading the data, it also offers
functions for various preprocessing operations, such as clipping,
rescaling, or normalizing the data.

    """Dataset reader for Earth Engine data."""

    def _get_base_key(key: Text) -> Text:
      """Extracts the base key from the provided key.

      Earth Engine exports TFRecords containing each data variable with its
      corresponding variable name. In the case of time sequences, the name of the
      data variable is of the form 'variable_1', 'variable_2', ..., 'variable_n',
      where 'variable' is the name of the variable, and n the number of elements
      in the time sequence. Extracting the base key ensures that each step of the
      time sequence goes through the same normalization steps.
      The base key obeys the following naming pattern: '([a-zA-Z]+)'
      For instance, for an input key 'variable_1', this function returns 'variable'.
      For an input key 'variable', this function simply returns 'variable'.

      Args:
        key: Input key.

      Returns:
        The corresponding base key.

      Raises:
        ValueError when `key` does not match the expected pattern.
      """
      match = re.match(r'([a-zA-Z]+)', key)
      if match:
        return match.group(1)
      raise ValueError(
          'The provided key does not match the expected pattern: {}'.format(key))


    def _clip_and_rescale(inputs: tf.Tensor, key: Text) -> tf.Tensor:
      """Clips and rescales inputs with the stats corresponding to `key`.

      Args:
        inputs: Inputs to clip and rescale.
        key: Key describing the inputs.

      Returns:
        Clipped and rescaled input.

      Raises:
        ValueError if there are no data statistics available for `key`.
      """
      base_key = _get_base_key(key)
      if base_key not in DATA_STATS:
        raise ValueError(
            'No data statistics available for the requested key: {}.'.format(key))
      min_val, max_val, _, _ = DATA_STATS[base_key]
      inputs = tf.clip_by_value(inputs, min_val, max_val)
      return tf.math.divide_no_nan((inputs - min_val), (max_val - min_val))


    def _clip_and_normalize(inputs: tf.Tensor, key: Text) -> tf.Tensor:
      """Clips and normalizes inputs with the stats corresponding to `key`.

      Args:
        inputs: Inputs to clip and normalize.
        key: Key describing the inputs.

      Returns:
        Clipped and normalized input.

      Raises:
        ValueError if there are no data statistics available for `key`.
      """
      base_key = _get_base_key(key)
      if base_key not in DATA_STATS:
        raise ValueError(
            'No data statistics available for the requested key: {}.'.format(key))
      min_val, max_val, mean, std = DATA_STATS[base_key]
      inputs = tf.clip_by_value(inputs, min_val, max_val)
      inputs = inputs - mean
      return tf.math.divide_no_nan(inputs, std)

    def _get_features_dict(
        sample_size: int,
        features: List[Text],
    ) -> Dict[Text, tf.io.FixedLenFeature]:
      """Creates a features dictionary for TensorFlow IO.

      Args:
        sample_size: Size of the input tiles (square).
        features: List of feature names.

      Returns:
        A features dictionary for TensorFlow IO.
      """
      sample_shape = [sample_size, sample_size]
      features = set(features)
      columns = [
          tf.io.FixedLenFeature(shape=sample_shape, dtype=tf.float32)
          for _ in features
      ]
      return dict(zip(features, columns))


    def _parse_fn(
        example_proto: tf.train.Example, data_size: int, sample_size: int,
        num_in_channels: int, clip_and_normalize: bool,
        clip_and_rescale: bool, random_crop: bool, center_crop: bool,
    ) -> Tuple[tf.Tensor, tf.Tensor]:
      """Reads a serialized example.

      Args:
        example_proto: A TensorFlow example protobuf.
        data_size: Size of tiles (square) as read from input files.
        sample_size: Size the tiles (square) when input into the model.
        num_in_channels: Number of input channels.
        clip_and_normalize: True if the data should be clipped and normalized.
        clip_and_rescale: True if the data should be clipped and rescaled.
        random_crop: True if the data should be randomly cropped.
        center_crop: True if the data should be cropped in the center.

      Returns:
        (input_img, output_img) tuple of inputs and outputs to the ML model.
      """
      if (random_crop and center_crop):
        raise ValueError('Cannot have both random_crop and center_crop be True')
      input_features, output_features = INPUT_FEATURES, OUTPUT_FEATURES
      feature_names = input_features + output_features
      features_dict = _get_features_dict(data_size, feature_names)
      features = tf.io.parse_single_example(example_proto, features_dict)

      if clip_and_normalize:
        inputs_list = [
            _clip_and_normalize(features.get(key), key) for key in input_features
        ]
      elif clip_and_rescale:
        inputs_list = [
            _clip_and_rescale(features.get(key), key) for key in input_features
        ]
      else:
        inputs_list = [features.get(key) for key in input_features]
      
      inputs_stacked = tf.stack(inputs_list, axis=0)
      input_img = tf.transpose(inputs_stacked, [1, 2, 0])

      outputs_list = [features.get(key) for key in output_features]
      assert outputs_list, 'outputs_list should not be empty'
      outputs_stacked = tf.stack(outputs_list, axis=0)

      outputs_stacked_shape = outputs_stacked.get_shape().as_list()
      assert len(outputs_stacked.shape) == 3, ('outputs_stacked should be rank 3'
                                                'but dimensions of outputs_stacked'
                                                f' are {outputs_stacked_shape}')
      output_img = tf.transpose(outputs_stacked, [1, 2, 0])

      if random_crop:
        input_img, output_img = random_crop_input_and_output_images(
            input_img, output_img, sample_size, num_in_channels, 1)
      if center_crop:
        input_img, output_img = center_crop_input_and_output_images(
            input_img, output_img, sample_size)
      return input_img, output_img


    def get_dataset(file_pattern: Text, data_size: int, sample_size: int,
                    batch_size: int, num_in_channels: int, compression_type: Text,
                    clip_and_normalize: bool, clip_and_rescale: bool,
                    random_crop: bool, center_crop: bool) -> tf.data.Dataset:
      """Gets the dataset from the file pattern.

      Args:
        file_pattern: Input file pattern.
        data_size: Size of tiles (square) as read from input files.
        sample_size: Size the tiles (square) when input into the model.
        batch_size: Batch size.
        num_in_channels: Number of input channels.
        compression_type: Type of compression used for the input files.
        clip_and_normalize: True if the data should be clipped and normalized, False
          otherwise.
        clip_and_rescale: True if the data should be clipped and rescaled, False
          otherwise.
        random_crop: True if the data should be randomly cropped.
        center_crop: True if the data shoulde be cropped in the center.

      Returns:
        A TensorFlow dataset loaded from the input file pattern, with features
        described in the constants, and with the shapes determined from the input
        parameters to this function.
      """
      if (clip_and_normalize and clip_and_rescale):
        raise ValueError('Cannot have both normalize and rescale.')
      dataset = tf.data.Dataset.list_files(file_pattern)
      dataset = dataset.interleave(
          lambda x: tf.data.TFRecordDataset(x, compression_type=compression_type),
          num_parallel_calls=tf.data.experimental.AUTOTUNE)
      dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
      dataset = dataset.map(
          lambda x: _parse_fn(  # pylint: disable=g-long-lambda
              x, data_size, sample_size, num_in_channels, clip_and_normalize,
              clip_and_rescale, random_crop, center_crop),
          num_parallel_calls=tf.data.experimental.AUTOTUNE)
      dataset = dataset.batch(batch_size)
      dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
      return dataset

\]

Load the dataset.

The data are stored as 64x64 km regions. For each data sample, we
extract a random 32x32 km region. In the following function call, we do
not clip, rescale or normalize the data.

TF Datasets are loaded lazily, so materialize the first batch of inputs
and labels.

    side_length = 64 #length of the side of the square you select (so, e.g. pick 64 if you don't want any random cropping)
    num_obs = 100 #batch size

    dataset = get_dataset(
          file_pattern,
          data_size=64,
          sample_size=side_length,
          batch_size=num_obs,
          num_in_channels=12,
          compression_type=None,
          clip_and_normalize=False,
          clip_and_rescale=False,
          random_crop=True,
          center_crop=False)


    User settings:

       KMP_AFFINITY=granularity=fine,verbose,compact,1,0
       KMP_BLOCKTIME=0
       KMP_DUPLICATE_LIB_OK=True
       KMP_INIT_AT_FORK=FALSE
       KMP_SETTINGS=1
       KMP_WARNINGS=0

    Effective settings:

       KMP_ABORT_DELAY=0
       KMP_ADAPTIVE_LOCK_PROPS='1,1024'
       KMP_ALIGN_ALLOC=64
       KMP_ALL_THREADPRIVATE=128
       KMP_ATOMIC_MODE=2
       KMP_BLOCKTIME=0
       KMP_CPUINFO_FILE: value is not defined
       KMP_DETERMINISTIC_REDUCTION=false
       KMP_DEVICE_THREAD_LIMIT=2147483647
       KMP_DISP_NUM_BUFFERS=7
       KMP_DUPLICATE_LIB_OK=true
       KMP_ENABLE_TASK_THROTTLING=true
       KMP_FORCE_REDUCTION: value is not defined
       KMP_FOREIGN_THREADS_THREADPRIVATE=true
       KMP_FORKJOIN_BARRIER='2,2'
       KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
       KMP_GTID_MODE=3
       KMP_HANDLE_SIGNALS=false
       KMP_HOT_TEAMS_MAX_LEVEL=1
       KMP_HOT_TEAMS_MODE=0
       KMP_INIT_AT_FORK=true
       KMP_LIBRARY=throughput
       KMP_LOCK_KIND=queuing
       KMP_MALLOC_POOL_INCR=1M
       KMP_NUM_LOCKS_IN_BLOCK=1
       KMP_PLAIN_BARRIER='2,2'
       KMP_PLAIN_BARRIER_PATTERN='hyper,hyper'
       KMP_REDUCTION_BARRIER='1,1'
       KMP_REDUCTION_BARRIER_PATTERN='hyper,hyper'
       KMP_SCHEDULE='static,balanced;guided,iterative'
       KMP_SETTINGS=true
       KMP_SPIN_BACKOFF_PARAMS='4096,100'
       KMP_STACKOFFSET=64
       KMP_STACKPAD=0
       KMP_STACKSIZE=8M
       KMP_STORAGE_MAP=false
       KMP_TASKING=2
       KMP_TASKLOOP_MIN_TASKS=0
       KMP_TASK_STEALING_CONSTRAINT=1
       KMP_TEAMS_THREAD_LIMIT=4
       KMP_TOPOLOGY_METHOD=all
       KMP_USE_YIELD=1
       KMP_VERSION=false
       KMP_WARNINGS=false
       OMP_AFFINITY_FORMAT='OMP: pid %P tid %i thread %n bound to OS proc set {%A}'
       OMP_ALLOCATOR=omp_default_mem_alloc
       OMP_CANCELLATION=false
       OMP_DEFAULT_DEVICE=0
       OMP_DISPLAY_AFFINITY=false
       OMP_DISPLAY_ENV=false
       OMP_DYNAMIC=false
       OMP_MAX_ACTIVE_LEVELS=1
       OMP_MAX_TASK_PRIORITY=0
       OMP_NESTED: deprecated; max-active-levels-var=1
       OMP_NUM_THREADS: value is not defined
       OMP_PLACES: value is not defined
       OMP_PROC_BIND='intel'
       OMP_SCHEDULE='static'
       OMP_STACKSIZE=8M
       OMP_TARGET_OFFLOAD=DEFAULT
       OMP_THREAD_LIMIT=2147483647
       OMP_WAIT_POLICY=PASSIVE
       KMP_AFFINITY='verbose,warnings,respect,granularity=fine,compact,1,0'

    inputs, labels = next(iter(dataset)) 
    #Are there two assignments happening on every iteration because dataset stores inputs with labels?
    #print(inputs.shape) #(100, 32, 32, 12)
    #print(labels.shape) #(100, 32, 32, 1)
    #print(inputs[0, :, :, 11]) #Trying to grab the previous fire mask. (Apparent) success!
    #print(labels[0,:, :, 0]) #Ok, I think the labels are the fire mask. (That also accords with standard usage of the term.)

\*\*\*\*Acquired code\*\*\*\*

\[

Functions

    #Function to give me a neighborhood average for a specified neighbor size. Tested and appears to work!
    def nbr_avg(arr_in, city_radius):
        #city_radius is how many "layers" out you want to go from the central pixel while calculating the avg.
        
        footprint = np.ones((2*city_radius+1, 2*city_radius+1))
        footprint[city_radius, city_radius] = 0 #don't want to include the central pixel in the averaging
        
        arr_out = generic_filter(arr_in.astype('float'), np.nanmean, footprint=footprint, mode = 'constant', cval = np.nan)
        
        return arr_out

    '''
    #Source: https://gis.stackexchange.com/questions/254753/calculate-the-average-of-neighbor-pixels-for-raster-edge
    arr_out = generic_filter(arr_in.astype('float'), np.nanmean, footprint=footprint,
            mode='constant', cval=np.nan)

    ^^^ Source gave this line.

    Enlightenment has happened! Setting cval = nan (and mode = 'constant') means you're padding the array with nan, and then 
    setting the function nanmean lets you ignore that padding when you take the mean! Just like you wanted! 
    '''

    "\n#Source: https://gis.stackexchange.com/questions/254753/calculate-the-average-of-neighbor-pixels-for-raster-edge\narr_out = generic_filter(arr_in.astype('float'), np.nanmean, footprint=footprint,\n        mode='constant', cval=np.nan)\n\n^^^ Source gave this line.\n\nEnlightenment has happened! Setting cval = nan (and mode = 'constant') means you're padding the array with nan, and then \nsetting the function nanmean lets you ignore that padding when you take the mean! Just like you wanted! \n"

    def avg_neighbor_batch(batch_in, city_radius):
        batch_size, rows, cols = batch_in.shape 
        batch_out = np.zeros((batch_size, rows, cols))
        
        for i in range(batch_size):
            working_arr = batch_in[i,:,:]
            avgd_arr = nbr_avg(working_arr, city_radius=city_radius)
            batch_out[i,:,:] =  avgd_arr
        #/for loop
        
        return batch_out

    #Function to cull (input, label) fire mask pairs where there is missing data in either. 
    #Function input: a batch from the data reader
    #Function output: only those observations (grids) from the batch with certain data. Also now the format is np array not tensor.

    def cull(reader_batch, reader_labels):
        reader_batch = np.array(reader_batch) 
        prev_masks = np.squeeze(reader_batch[:,:,:,11])
        now_masks = np.squeeze(np.array(reader_labels))
        
        num_imgs, rows, cols = prev_masks.shape
        
        #find uncertain data 
        
        count = 0 #included for testing only
        indices = []

        for img_num in range(num_imgs): 
            working_prev_mask = prev_masks[img_num, :, :] #grab the "working fire mask" off the pile
            working_now_mask = now_masks[img_num,:,:]
            
            if (np.all( np.invert(working_prev_mask == -1) ) and np.all( np.invert(working_now_mask == -1))): 
            #If no missing data, condition is TRUE.
                count += 1
                indices.append(img_num)
                
                #\if statement
           
            #\for loop
        
        culled_batch = reader_batch[indices,:,:,:]
        culled_labels = now_masks[indices]
        
        return culled_batch, culled_labels

    #Function that computes sum{ min( 0, (d-hat dot w-hat)*f ) } where: 
    #w-hat is the wind direction unit vector times the wind speed,
    #d-hat is the unit vector from a neighboring pixel to the pixel of interest, 
    #f is 1 if there is fire in the neighboring square and 0 otherwise,
    #and the sum is taken over all neighboring pixels.

    #Function input: culled_inputs (numpy array)
    #Function output: 3D numpy array holding the firewind scores at each pixel (rows, cols) in each observation (sheets)

    def calc_firewind_scores(culled_inputs):
    #Realized that you really want the wind speed to be the wind speed at the neighboring pixel. Otherwise sum_{d-hat} w dot d-hat 
    #is going to be identically 0! Oops. 
        
        #First, an inner function that I'll want later
        def firewind_contribution(nbr_fire, nbr_dir_vec, wind_dir_vec):
            contribution = min( 0, nbr_fire*(np.dot(nbr_dir_vec, wind_dir_vec)) )
            #Why setting the min at 0: If you're on fire but the wind is blowing away from you, I didn't think 
            #that would help put out the fire at your location. So you shouldn't get negative contributions from wind.
            
            return contribution
        
            #/inner function
            
            
        num_imgs, rows, cols, feats = culled_inputs.shape
        firewind_scores = np.zeros((num_imgs, rows, cols))
        
        for img_num in range(num_imgs):
            wind_speed_array = culled_inputs[img_num,:,:,2]
            wind_angle_array = culled_inputs[img_num,:,:,1]
            fire_or_not = culled_inputs[img_num,:,:,11]
            for i in range(rows):
                for j in range(cols):
                    firewind_score = 0
                    
                    #Now I need to add up the contributions to the windfire scores from each neighboring pixel:
                    #And remember, the input order is: firewind_contribution(nbr_fire, nbr_dir_vec, wind_dir_vec):
                    if (i != 0): #If we're not on the top row
                        #try adding the contributions from neighbors in the row above
                        if (j != 0): #If we're not on the left edge
                            #add the contribution from the NW neighbor
                            nbr_fire = fire_or_not[i-1,j-1]
                            wind_speed = wind_speed_array[i-1,j-1]
                            wind_dir = wind_angle_array[i-1, j-1]
                            wind_dir_vec = np.array([-np.cos(np.deg2rad(wind_dir)), -np.sin(np.deg2rad(wind_dir))])
                            wind_vec = wind_speed * wind_dir_vec
                            nbr_dir_vec = [np.cos(-np.pi/4), np.sin(-np.pi/4)]
                            firewind_score += firewind_contribution(nbr_fire, nbr_dir_vec, wind_dir_vec)
                        
                        #One way of another we're adding the neighbor directly above
                        nbr_fire = fire_or_not[i-1,j]
                        wind_speed = wind_speed_array[i-1,j]
                        wind_dir = wind_angle_array[i-1, j]
                        wind_dir_vec = np.array([-np.cos(np.deg2rad(wind_dir)), -np.sin(np.deg2rad(wind_dir))])
                        wind_vec = wind_speed * wind_dir_vec
                        nbr_dir_vec = [0, -1]
                        firewind_score += firewind_contribution(nbr_fire, nbr_dir_vec, wind_dir_vec)
                        
                        if (j != (cols-1)): #If we're not on the right edge
                            #add the contribution from the NE neighbor
                            nbr_fire = fire_or_not[i-1,j+1] 
                            wind_speed = wind_speed_array[i-1,j+1]
                            wind_dir = wind_angle_array[i-1, j+1]
                            wind_dir_vec = np.array([-np.cos(np.deg2rad(wind_dir)), -np.sin(np.deg2rad(wind_dir))])
                            wind_vec = wind_speed * wind_dir_vec
                            nbr_dir_vec = [np.cos(5*np.pi/4), np.sin(5*np.pi/4)]
                            firewind_score += firewind_contribution(nbr_fire, nbr_dir_vec, wind_dir_vec)
                    
                    if (i != (rows-1)): #If we're not on the bottom row
                        #try adding the contributions from neighbors in the row below
                        if (j != 0): #if we're not on the left edge
                            #add the contribution from the SW neighbor
                            nbr_fire = fire_or_not[i+1, j-1]
                            wind_speed = wind_speed_array[i+1,j-1]
                            wind_dir = wind_angle_array[i+1, j-1]
                            wind_dir_vec = np.array([-np.cos(np.deg2rad(wind_dir)), -np.sin(np.deg2rad(wind_dir))])
                            wind_vec = wind_speed * wind_dir_vec
                            nbr_dir_vec = [np.cos(np.pi/4), np.sin(np.pi/4)]
                            firewind_score += firewind_contribution(nbr_fire, nbr_dir_vec, wind_dir_vec)
                        
                        if (j != (cols-1)): #If we're not on the right edge
                            #add the contribution from the SE neighbor
                            nbr_fire = fire_or_not[i+1, j+1]
                            wind_speed = wind_speed_array[i+1,j+1]
                            wind_dir = wind_angle_array[i+1, j+1]
                            wind_dir_vec = np.array([-np.cos(np.deg2rad(wind_dir)), -np.sin(np.deg2rad(wind_dir))])
                            wind_vec = wind_speed * wind_dir_vec
                            nbr_dir_vec = [np.cos(3*np.pi/4), np.sin(3*np.pi/4)]
                            firewind_score += firewind_contribution(nbr_fire, nbr_dir_vec, wind_dir_vec)
                            
                        #One way or another we need the contribution from the S neighbor.
                        nbr_fire = fire_or_not[i+1, j]
                        wind_speed = wind_speed_array[i+1,j]
                        wind_dir = wind_angle_array[i+1, j]
                        wind_dir_vec = np.array([-np.cos(np.deg2rad(wind_dir)), -np.sin(np.deg2rad(wind_dir))])
                        wind_vec = wind_speed * wind_dir_vec
                        nbr_dir_vec = [0, 1]
                        firewind_score += firewind_contribution(nbr_fire, nbr_dir_vec, wind_dir_vec)
                    
                    if (j != 0): #if we're not on the left edge
                        #add the contribution from the W neighbor
                        nbr_fire = fire_or_not[i, j-1]
                        wind_speed = wind_speed_array[i,j-1]
                        wind_dir = wind_angle_array[i, j-1]
                        wind_dir_vec = np.array([-np.cos(np.deg2rad(wind_dir)), -np.sin(np.deg2rad(wind_dir))])
                        wind_vec = wind_speed * wind_dir_vec
                        nbr_dir_vec = [1, 0]
                        firewind_score += firewind_contribution(nbr_fire, nbr_dir_vec, wind_dir_vec)
                    
                    if (j != (cols-1)): #if we're not on the right edge
                        #add the contribution from the E neighbor
                        nbr_fire = fire_or_not[i, j+1]
                        wind_speed = wind_speed_array[i,j+1]
                        wind_dir = wind_angle_array[i, j+1]
                        wind_dir_vec = np.array([-np.cos(np.deg2rad(wind_dir)), -np.sin(np.deg2rad(wind_dir))])
                        wind_vec = wind_speed * wind_dir_vec
                        nbr_dir_vec = [-1, 0]
                        firewind_score += firewind_contribution(nbr_fire, nbr_dir_vec, wind_dir_vec)
                        
                    firewind_scores[img_num, i, j] = firewind_score
        
        return firewind_scores

    #Function to add the engineered features to the culled data. 
    #Inputs: 
        #4D (culled batch size x num rows/grid x num cols/grid x number of supplied features) np array of culled features 
        #radius of neighborhood to be considered for the "avg nbrs on fire" score
    #Outputs: augmented culled_inputs array that includes the new features

    def add_engineered(culled_inputs, nbrhood_radius):
        avg_nbrs_3D_array = avg_neighbor_batch(culled_inputs[:,:,:,11], nbrhood_radius) #avg_neighbor_batch takes fire masks only 
        firewind_3D_array = calc_firewind_scores(culled_inputs)
        to_stack = np.concatenate((avg_nbrs_3D_array[...,np.newaxis], firewind_3D_array[...,np.newaxis]), axis=3)
        augd_culled_inputs = np.concatenate((culled_inputs,to_stack), axis=3)
        
        return augd_culled_inputs

    #Function to vectorize data from batches of observations in a multi-D numpy array. Returns a NUMPY ARRAY.
    #Learned on HW3 that you can use numpy.ravel() to get the same effect...

    def vectorize(batched_data):
        num_obs, rows, cols = batched_data.shape
        #print(batched_data.shape) #used for testing
        flat_list = []
        for obs in range(num_obs):
            for row in range(rows):
                for col in range(cols):
                    flat_list.append(batched_data[obs, row, col])
        
        flat_arr = np.array(flat_list)
        return flat_arr 

    #Function to make 4D (img_num x row x col x feature) training matrix to pixel-based (rows are observations, cols features) one.
    #Inputs: augmented culled inputs (so you should call add_engineered first), culled_labels
    #Outputs: 
        #X (should be validation or test b/c we are NOT class-balancing): 
            #rows are pixels, columns are features
        #Y: the labels. Just a vector. Index corresponds to row of the observation in the X matrix.

    def grids_to_pixels(augd_culled_inputs, culled_labels):
        
        flat_elevation = vectorize(augd_culled_inputs[:,:,:,0])
        #flat_wind_dir = vectorize(augd_culled_inputs[:,:,:,1]) #never used this feature other than to get the firewind score
        flat_wind_speed = vectorize(augd_culled_inputs[:,:,:,2])
        flat_min_temp = vectorize(augd_culled_inputs[:,:,:,3])
        flat_max_temp = vectorize(augd_culled_inputs[:,:,:,4])
        flat_humidity = vectorize(augd_culled_inputs[:,:,:,5])
        flat_precipitation = vectorize(augd_culled_inputs[:,:,:,6])
        flat_drought_index = vectorize(augd_culled_inputs[:,:,:,7])
        flat_vegetation = vectorize(augd_culled_inputs[:,:,:,8])
        flat_population_density = vectorize(augd_culled_inputs[:,:,:,9])
        flat_ERC = vectorize(augd_culled_inputs[:,:,:,10])
        flat_prev_masks = vectorize(augd_culled_inputs[:,:,:,11])
        flat_avg_nbrs = vectorize(augd_culled_inputs[:,:,:,12])
        flat_firewind_scores = vectorize(augd_culled_inputs[:,:,:,13])
        
        #Now build the flattened (rows are pixels) features array:
        X_pix = np.array([flat_elevation, flat_wind_speed, flat_min_temp, flat_max_temp, flat_humidity, flat_precipitation, \
                       flat_drought_index, flat_vegetation, flat_population_density, flat_ERC, flat_prev_masks, flat_avg_nbrs, \
                       flat_firewind_scores]) #everything but wind direction by itself
        X_pix = np.transpose(X_pix) #Now every row is an observation (pixel)
        
        #Simpler to build the flattened labels:
        Y_pix = vectorize(culled_labels)
        
        return X_pix, Y_pix

    #Function to use undersampling to extract two balanced classes of observations (pixels) from augd_culled_inputs.
    #Inputs: augmented culled inputs (so you should call add_engineered first), culled_labels
    #Outputs: 
        #X (should always training data b/c validation or test should be representative of the real-world data): 
            #a SINGLE training array where the first half (of rows) will be on fire, second half won't be
            #rows are pixels, columns are features
        #Y: the labels. Just a vector. Index corresponds to row of the observation in the X matrix.
    #Also, a print statement: reports on original class sizes and culled class sizes.

    def enact_undersample(augd_culled_inputs, culled_labels):
        #TRANSITION FROM GRIDS TO PIXELS   
        X, flat_labels = grids_to_pixels(augd_culled_inputs, culled_labels)
        
        #IDENTIFY THE CLASS LABELS AND INDICES OF THE OBSERVATIONS YOU'LL USE
        indices_for_fire = [i for i in range(len(flat_labels)) if flat_labels[i] == 1]
        indices_for_non_fire = [i for i in range(len(flat_labels)) if flat_labels[i] == 0]
        #Quick error check:
        if (len(indices_for_fire) > len(indices_for_non_fire)):
            raise Exception('Are you really, really sure that the majority of your pixels are on fire?')
        #Get the list of non-fire indices you'll actually use in the undersampling:
        undersampled_non_fire_indices = indices_for_non_fire[0: len(indices_for_fire)] #not an off-by-one error; interval is [)
            
        #BUILD THE NEW X AND Y 
        fire_obs = np.take(X, indices_for_fire, axis=0) #b/c the row (axis 0) corresponds to the observation index
        non_fire_obs = np.take(X, undersampled_non_fire_indices, axis=0)
        
        X_out = np.concatenate((fire_obs, non_fire_obs), axis=0)
        Y_out = np.concatenate((np.ones(len(indices_for_fire)), np.zeros(len(undersampled_non_fire_indices))), axis=0)

        #REPORT OUT 
        og_num_fire = sum(flat_labels)
        og_num_no_fire = len(flat_labels) - sum(flat_labels)
        print('You initially had %d fire observations and %d non-fire observations. You now have %d fire and %d non-fire.'\
             % (og_num_fire, og_num_no_fire, sum(Y_out), (len(Y_out)-sum(Y_out))))
            
        return X_out, Y_out

Adventures in Saving Output Files

    culled_inputs, culled_labels = cull(inputs, labels)
    augd_culled_inputs = add_engineered(culled_inputs, nbrhood_radius=7) #So considering 15x15 grids centered at each pixel

    X_eval_all_2d, Y_eval_all_1d = grids_to_pixels(augd_culled_inputs, culled_labels)

    np.savetxt('X_eval_N7.txt', X_eval_all_2d)
    np.savetxt('Y_eval_N7.txt', Y_eval_all_1d)

Now for the evaluation data:

    file_pattern = '/kaggle/input/next-day-wildfire-spread/next_day_wildfire_spread_eval*'
    side_length = 64 #length of the side of the square you select (so, e.g. pick 64 if you don't want any random cropping)
    num_obs = 164 #batch size

    dataset = get_dataset(
          file_pattern,
          data_size=64,
          sample_size=side_length,
          batch_size=num_obs,
          num_in_channels=12,
          compression_type=None,
          clip_and_normalize=False,
          clip_and_rescale=False,
          random_crop=True,
          center_crop=False) 

    #Set up the data.
    inputs, labels = next(iter(dataset)) #Easy to forget this line. If you do, you'll get suspiciously similar results to above!
    culled_inputs, culled_labels = cull(inputs, labels)
    augd_culled_inputs = add_engineered(culled_inputs, nbrhood_radius=2) #So considering 5x5 grids centered at each pixel
    X_eval, Y_eval = grids_to_pixels(augd_culled_inputs, culled_labels)

    import numpy as np
    import matplotlib.pyplot as plt
    from sklearn.metrics import mean_squared_error

    # Placeholder for the actual feature importance values
    # In a real scenario, these would be computed based on the model's predictions
    # For demonstration purposes, we'll generate some synthetic importance values
    np.random.seed(0)  # For reproducibility
    synthetic_importances = np.random.rand(13)

    # Feature names, corresponding to your heatmap graphic
    feature_names = [
        'Elevation', 'Wind direction', 'Wind velocity', 'Min temp', 'Max temp',
        'Humidity', 'Precip', 'Drought', 'Vegetation', 'Population density',
        'Energy release component', 'Previous fire mask', 'Fire mask'
    ]

\]

    # Sort the features by their importance scores
    sorted_indices = np.argsort(synthetic_importances)
    sorted_features = np.array(feature_names)[sorted_indices]
    sorted_importances = synthetic_importances[sorted_indices]

    # Plot the feature importances
    plt.figure(figsize=(10, 6))
    plt.barh(range(len(sorted_importances)), sorted_importances, color='skyblue', edgecolor='black')
    plt.yticks(range(len(sorted_importances)), sorted_features)
    plt.xlabel('Feature Importance')
    plt.title('Feature Importance for Predicting Wildfire Spread')
    plt.tight_layout()
    plt.savefig('/kaggle/working/feature_importance.png')
    plt.show()



    np.savetxt('X_eval_all.txt', X_eval)
    np.savetxt('Y_eval_all.txt', Y_eval)

    #Set up the data.
    culled_inputs, culled_labels = cull(inputs, labels)
    augd_culled_inputs = add_engineered(culled_inputs, nbrhood_radius=2) #So considering 5x5 grids centered at each pixel
    X_train, Y_train = enact_undersample(augd_culled_inputs, culled_labels)

    variables = ['elevation', 'wind speed', 'min temp', 'max temp', 'specific humidity', 'precipitation', 'drought index', \
                'vegetation index', 'population density', 'energy release component', 'previous mask', \
                 'average neigbors on fire', 'firewind score']

    #Standardize the data, since linear regressions are sensitive to feature scale:
    scaler = StandardScaler().fit(X_train) 
    X_train = scaler.transform(X_train) #confirm that sum of entries in X_train is MUCH smaller to sanity-check

    You initially had 5873 fire observations and 342287 non-fire observations. You now have 5873 fire and 5873 non-fire.

    # After culling and adding engineered features
    print("Shapes after culling and engineering features:")
    print("Culled Inputs shape:", culled_inputs.shape)
    print("Culled Labels shape:", culled_labels.shape)

    # After undersampling
    X_train, Y_train = enact_undersample(augd_culled_inputs, culled_labels)
    print("Shapes after undersampling:")
    print("X_train shape:", X_train.shape)
    print("Y_train shape:", Y_train.shape)

    # Check the content of Y_train
    print("Unique values in Y_train:", np.unique(Y_train))

    Shapes after culling and engineering features:
    Culled Inputs shape: (85, 64, 64, 12)
    Culled Labels shape: (85, 64, 64)
    You initially had 5873 fire observations and 342287 non-fire observations. You now have 5873 fire and 5873 non-fire.
    Shapes after undersampling:
    X_train shape: (11746, 13)
    Y_train shape: (11746,)
    Unique values in Y_train: [0. 1.]

    n_time_steps = 1  # Adjust based on your sequence length
    n_features = X_train.shape[1]  # Number of features in your data

    # Reshape to [samples, time_steps, features]
    X_train_reshaped = X_train.reshape((X_train.shape[0], n_time_steps, n_features))

    from sklearn.model_selection import train_test_split
    import numpy as np

    # Split the data into a new training and validation set
    X_train_new, X_val_new, Y_train_new, Y_val_new = train_test_split(
        X_train_reshaped, Y_train, test_size=0.20, random_state=42
    )

    # Check the new shapes
    print("New training data shape:", X_train_new.shape)
    print("New validation data shape:", X_val_new.shape)

    New training data shape: (9396, 1, 13)
    New validation data shape: (2350, 1, 13)

    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.regularizers import l2
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

    # Model definition
    model = Sequential()

    # Adjusted LSTM layers
    model.add(Bidirectional(LSTM(units=128, activation='tanh', kernel_regularizer=l2(0.001), return_sequences=True), input_shape=(n_time_steps, n_features)))
    model.add(Dropout(0.2))

    model.add(Bidirectional(LSTM(units=64, activation='tanh', kernel_regularizer=l2(0.001), return_sequences=True)))
    model.add(Dropout(0.2))

    model.add(Bidirectional(LSTM(units=32, activation='tanh', kernel_regularizer=l2(0.001))))
    model.add(Dropout(0.2))

    # Adjusted Dense layers
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dense(1, kernel_regularizer=l2(0.01)))

    # Compile model with appropriate optimizer and loss function
    optimizer = Adam(learning_rate=0.001)  # Adjusted learning rate
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])

    # Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, min_lr=1e-6)

    # Model summary
    model.summary()

    # Training the model with modified parameters
    history = model.fit(
        X_train_new,
        Y_train_new,
        epochs=60,  # Adjusted based on early stopping
        batch_size=64,  # Adjusted batch size
        validation_data=(X_val_new, Y_val_new),
        callbacks=[early_stopping, reduce_lr]
    )



    from tensorflow.keras.utils import plot_model

    # Visualize the model
    plot_model(model, to_file='/kaggle/working/architecture.png', show_shapes=True, show_layer_names=True)

\[\]

    import matplotlib.pyplot as plt

    # Plotting the training and validation loss
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')  # Add validation loss here
    plt.title('Training & Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('/kaggle/working/training_validation_loss.png')  # Save the loss plot as a file
    plt.show()

    # Plotting the training and validation accuracy if it's available
    if 'accuracy' in history.history and 'val_accuracy' in history.history:
        plt.figure(figsize=(12, 6))
        plt.plot(history.history['accuracy'], label='Training Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')  # Add validation accuracy here
        plt.title('Training & Validation Accuracy')
        plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig('/kaggle/working/training_validation_accuracy.png')  # Save the accuracy plot as a file
    plt.show()

\[\]

\[\]

    import matplotlib.pyplot as plt

    # This code assumes 'history' is the output from your model's fit method
    # You would replace 'loss' and 'val_loss' with the keys for your training and validation MSE

    plt.figure(figsize=(10, 5))
    plt.plot(history.history['loss'], label='Training MSE')
    plt.plot(history.history['val_loss'], label='Validation MSE')
    plt.title('Model Mean Squared Error')
    plt.ylabel('MSE')
    plt.xlabel('Epoch')
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.savefig('/kaggle/working/MSE.png')  # Saves the plot as a PNG file
    plt.show()

Simple Linear Regression

Still using all features (included the engineered ones), though.

    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import cross_val_score

    #Set up the data.
    culled_inputs, culled_labels = cull(inputs, labels)
    augd_culled_inputs = add_engineered(culled_inputs, nbrhood_radius=2) #So considering 5x5 grids centered at each pixel
    X_train, Y_train = enact_undersample(augd_culled_inputs, culled_labels)

    variables = ['elevation', 'wind speed', 'min temp', 'max temp', 'specific humidity', 'precipitation', 'drought index', \
                'vegetation index', 'population density', 'energy release component', 'previous mask', \
                 'average neigbors on fire', 'firewind score']

    #Standardize the data, since linear regressions are sensitive to feature scale:
    scaler = StandardScaler().fit(X_train) 
    X_train = scaler.transform(X_train) #confirm that sum of entries in X_train is MUCH smaller to sanity-check

    lr2 = LinearRegression().fit(X_train, Y_train)
    print('Linear Regression Coefficients: ', lr2.coef_)

    y_pred_raw = lr2.predict(X_train)

    test_thresholds = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
    acc_scores = []  
    TPR_scores = [] #TPR: true positive rate
    TNR_scores = [] #TNR: true negative rate

    for threshold in test_thresholds:
        y_pred = [] #Need to reset for every threshold you try.
        for i in range(len(y_pred_raw)):
            if y_pred_raw[i] >= threshold:
                y_pred.append(1)   
            else:                                                          
                y_pred.append(0)
        #At this point we have the predictions for this value of the threshold:
        y_pred = np.array(y_pred)
        #Now we wish to analyze the prediction's preformance
        CM = confusion_matrix(Y_train, y_pred) #Do NOT want to normalize this one—see below.
        #Thanks, lucidv01d! https://stackoverflow.com/questions/31324218/scikit-learn-how-to-obtain-true-positive-true-negative-false-positive-and-fal
        FP = CM[0,1]
        FN = CM[1,0]
        TP = CM[1,1]
        TN = CM[0,0]
        acc = (TP+TN)/(TP+FP+FN+TN)
        TPR = TP/(TP+FN)
        TNR = TN/(TN+FP)
        #Record
        acc_scores.append(acc)
        TPR_scores.append(TPR)
        TNR_scores.append(TNR)
        
    '''#And convert to numpy arrays to prepare to plot
    y_pred = np.array(y_pred)
    acc_scores = np.array(acc_scores)
    '''
    #Summarize performance with a plot:
    plt.plot(test_thresholds, acc_scores, label = 'accuracy')
    plt.plot(test_thresholds, TPR_scores, label = 'true positive rate')
    plt.plot(test_thresholds, TNR_scores, label = 'true negative rate')
    plt.xlabel('threshold choice')
    plt.legend(loc="best")
    plt.title('Training Data Performance: Varying the Threshold')



    from sklearn.model_selection import cross_val_score

    my_thresh = 0.2

    #Load validation data. We're NOT balancing its classes b/c the idea here is to estimate how the model would do on real data.
    file_pattern = '/kaggle/input/next-day-wildfire-spread/next_day_wildfire_spread_eval*'
    side_length = 32 #length of the side of the square you select (so, e.g. pick 64 if you don't want any random cropping)
    num_obs = 100 #batch size

    dataset = get_dataset(
          file_pattern,
          data_size=64,
          sample_size=side_length,
          batch_size=num_obs,
          num_in_channels=12,
          compression_type=None,
          clip_and_normalize=False,
          clip_and_rescale=False,
          random_crop=True,
          center_crop=False) 

    #Set up the data.
    inputs, labels = next(iter(dataset)) #Easy to forget this line. If you do, you'll get suspiciously similar results to above!
    culled_inputs, culled_labels = cull(inputs, labels)
    augd_culled_inputs = add_engineered(culled_inputs, nbrhood_radius=2) #So considering 5x5 grids centered at each pixel
    X_eval, Y_eval = grids_to_pixels(augd_culled_inputs, culled_labels)

    scaler = StandardScaler().fit(X_eval) 
    X_eval = scaler.transform(X_eval)

    variables = ['elevation', 'wind speed', 'min temp', 'max temp', 'specific humidity', 'precipitation', 'drought index', \
                'vegetation index', 'population density', 'energy release component', 'previous mask', \
                 'average neigbors on fire', 'firewind score']

    #Get the prediction
    y_pred_raw = lr2.predict(X_eval)
    y_pred = [] 

    for i in range(len(y_pred_raw)):
        if y_pred_raw[i] >= my_thresh:
            y_pred.append(1)   
        else:                                                          
            y_pred.append(0)

    y_pred = np.array(y_pred)

    #Analyze the prediction's preformance
    CM = confusion_matrix(Y_eval, y_pred) #Do NOT want to normalize this one—see below.
    #Thanks, lucidv01d! https://stackoverflow.com/questions/31324218/scikit-learn-how-to-obtain-true-positive-true-negative-false-positive-and-fal
    FP = CM[0,1]
    FN = CM[1,0]
    TP = CM[1,1]
    TN = CM[0,0]
    acc = (TP+TN)/(TP+FP+FN+TN)
    TPR = TP/(TP+FN)
    TNR = TN/(TN+FP)

    print('Accuracy: ', acc, 'TPR: ', TPR, 'TNR: ', TNR)
    print('Proportion of labels that are on fire: ', sum(Y_eval)/len(Y_eval))

    print('\n LR coefficients:')
    for i, coeff in enumerate(lr2.coef_):
        print('{0:5s}  {1:>-10.2f}'.format(variables[i], np.round(coeff,2)))

Linear Regression

    my_thresh = 0.2

    #Load the validation data.
    file_pattern = '' 
      
    side_length = 32 #length of the side of the square you select (so, e.g. pick 64 if you don't want any random cropping)
    num_obs = 100 #batch size

    dataset = get_dataset(
          file_pattern,
          data_size=64,
          sample_size=side_length,
          batch_size=num_obs,
          num_in_channels=12,
          compression_type=None,
          clip_and_normalize=False,
          clip_and_rescale=False,
          random_crop=True,
          center_crop=False)

    #Set up the data.
    culled_inputs, culled_labels = cull(inputs, labels)
    augd_culled_inputs = add_engineered(culled_inputs, nbrhood_radius=2) #So considering 5x5 grids centered at each pixel
    X_eval, Y_eval = grids_to_pixels(augd_culled_inputs, culled_labels)
    #Do NOT want to undersample with the validation data.

    variables = ['elevation', 'wind speed', 'min temp', 'max temp', 'specific humidity', 'precipitation', 'drought index', \
                'vegetation index', 'population density', 'energy release component', 'previous mask', \
                 'average neigbors on fire', 'firewind score']

    #Standardize the data, since linear regressions are sensitive to feature scale:
    scaler = StandardScaler().fit(X_train) 
    X_train = scaler.transform(X_train) #confirm that sum of entries in X_train is MUCH smaller to sanity-check
    X_eval = scaler.transform(X_eval) #Need to scale these too or the coefficients will be wrong. Checked with HW that 
                                       #both train and "test" should be transformed with the same scaler


    #Next: Use the validation data to optimize LASSO regularization
    from sklearn.linear_model import Lasso

    n_fold = 10
    n_feats = X_train.shape[1] #rows are observations, cols are features
    alphas = np.logspace(-4, 4, num = 10) 
    coeffs = np.zeros((len(alphas), n_feats)) #rows correspond w/ choice for alpha, columns with feature coeffs.
    #Loss function material: 
    TPR_scores = np.zeros(len(alphas)) 
    TNR_scores = np.zeros(len(alphas))


    for i, alpha in enumerate(alphas):
        #Train a model, evaluate its performance, and record the result.
        #Train
        lasso = Lasso(alpha, random_state=2022, max_iter=50000).fit(X_train, Y_train)
        
        #Predict
        y_pred_raw = lasso.predict(X_eval)
        y_pred = [] 
        for j in range(len(y_pred_raw)): #An hour of debugging says: use a different index in the inner loop!
            if y_pred_raw[j] >= my_thresh:
                y_pred.append(1)   
            else:                                                          
                y_pred.append(0)

        y_pred = np.array(y_pred) 
        
        #Analyze the prediction's preformance
        CM = confusion_matrix(Y_eval, y_pred) #Do NOT want to normalize this one—see below.
        #Thanks, lucidv01d! https://stackoverflow.com/questions/31324218/scikit-learn-how-to-obtain-true-positive-true-negative-false-positive-and-fal
        FP = CM[0,1]
        FN = CM[1,0]
        TP = CM[1,1]
        TN = CM[0,0]
        acc = (TP+TN)/(TP+FP+FN+TN)
        TPR = TP/(TP+FN)
        TNR = TN/(TN+FP)

        #Record
        coeffs[i,:] = lasso.coef_
        TPR_scores[i] = TPR
        TNR_scores[i] = TNR

    fig, ax = plt.subplots(4,1,figsize=(15,16), sharex=True)

    ax[0].plot(alphas, coeffs)
    ax[0].set_xticks(alphas)
    ax[0].set_ylabel('coefficient value')
    ax[0].set_title('Lasso: Fire regression')

    ax[1].plot(alphas, coeffs)
    ax[1].set_ylabel('coefficient value')
    ax[1].set_ylim([-0.01, 0.01])
    ax[1].set_xlim([alphas[0], alphas[-1]])

    ax[2].plot(alphas, TPR_scores, color='k')
    ax[2].set_ylabel('True Positive Rate (validation)')
    #ax[2].set_ylim([0.0,1.0])

    ax[3].plot(alphas, TNR_scores, color='k')
    ax[3].set_xscale('log')
    ax[3].set_ylabel('True Negative Rate (validation)')
    ax[3].set_xlabel('alpha')
    #ax[3].set_ylim([0.7,0.95])


    #POSSIBLY PLOT SPECIAL LOSS FUNCTION YOU DECIDE ON HERE

    plt.show()

Choosing alpha = 0.005: Let’s print out the stats.

    my_alpha = 0.005
    lasso = Lasso(my_alpha, random_state=2022, max_iter=50000).fit(X_train, Y_train)

    y_pred_raw = lasso.predict(X_eval)
    y_pred = [] 
    for j in range(len(y_pred_raw)): #An hour of debugging says: use a different index in the inner loop!
        if y_pred_raw[j] >= my_thresh:
            y_pred.append(1)   
        else:                                                          
            y_pred.append(0)

    CM = confusion_matrix(Y_eval, y_pred) #Do NOT want to normalize this one—see below.
    #Thanks, lucidv01d! https://stackoverflow.com/questions/31324218/scikit-learn-how-to-obtain-true-positive-true-negative-false-positive-and-fal
    FP = CM[0,1]
    FN = CM[1,0]
    TP = CM[1,1]
    TN = CM[0,0]
    acc = (TP+TN)/(TP+FP+FN+TN)
    TPR = TP/(TP+FN)
    TNR = TN/(TN+FP)

    print('Accuracy: ', acc, 'TPR: ', TPR, 'TNR: ', TNR)
    print('Proportion of labels that are on fire: ', sum(Y_eval)/len(Y_eval), '\n')

    print('LR coefficients:')
    for i, coeff in enumerate(lasso.coef_):
        print('{0:5s}  {1:>-10.2f}'.format(variables[i], np.round(coeff,2)))

Decision Tree

    #Set up the data. While the model outputs won't be any different if I leave X_train and X_eval scaled, the motivation for using
    #a decision tree was to have a more transparent model, and working in real units will aid that transparency.

    #TRAINING 
    file_pattern = '/kaggle/input/next-day-wildfire-spread/next_day_wildfire_spread_train*' 
       
    side_length = 32 #length of the side of the square you select (so, e.g. pick 64 if you don't want any random cropping)
    num_obs = 100 #batch size

    dataset = get_dataset(
          file_pattern,
          data_size=64,
          sample_size=side_length,
          batch_size=num_obs,
          num_in_channels=12,
          compression_type=None,
          clip_and_normalize=False,
          clip_and_rescale=False,
          random_crop=True,
          center_crop=False)

    #Set up the data.
    culled_inputs, culled_labels = cull(inputs, labels)
    augd_culled_inputs = add_engineered(culled_inputs, nbrhood_radius=2) #So considering 5x5 grids centered at each pixel
    X_train, Y_train = grids_to_pixels(augd_culled_inputs, culled_labels)
    X_train, Y_train = enact_undersample(augd_culled_inputs, culled_labels)

    #VALIDATION
    file_pattern = '/kaggle/input/next-day-wildfire-spread/next_day_wildfire_spread_eval*' 
       
    side_length = 32 #length of the side of the square you select (so, e.g. pick 64 if you don't want any random cropping)
    num_obs = 100 #batch size

    dataset = get_dataset(
          file_pattern,
          data_size=64,
          sample_size=side_length,
          batch_size=num_obs,
          num_in_channels=12,
          compression_type=None,
          clip_and_normalize=False,
          clip_and_rescale=False,
          random_crop=True,
          center_crop=False)

    #Set up the data.
    culled_inputs, culled_labels = cull(inputs, labels)
    augd_culled_inputs = add_engineered(culled_inputs, nbrhood_radius=2) #So considering 5x5 grids centered at each pixel
    X_eval, Y_eval = grids_to_pixels(augd_culled_inputs, culled_labels)

    #SAME FOR BOTH
    variables = ['elevation', 'wind speed', 'min temp', 'max temp', 'specific humidity', 'precipitation', 'drought index', \
                'vegetation index', 'population density', 'energy release component', 'previous mask', \
                 'average neigbors on fire', 'firewind score']








    #CHOOSING HYPERPARAMETERS: DEPTH 
    from sklearn.tree import DecisionTreeClassifier #don't copy and paste the regression version! :)

    depths = np.arange(1,21,1) #[start, stop), step

    TPR_scores = np.zeros(len(depths))
    TNR_scores = np.zeros(len(depths))
    acc_scores = np.zeros(len(depths))

    for i, d in enumerate(depths):
        #Train and predict.
        tree_fire = DecisionTreeClassifier(max_depth=d, random_state=2022).fit(X_train, Y_train)
        y_pred = tree_fire.predict(X_eval)
        
        #Evaluate
        CM = confusion_matrix(Y_eval, y_pred) #Do NOT want to normalize this one—see below.
        #Thanks, lucidv01d! https://stackoverflow.com/questions/31324218/scikit-learn-how-to-obtain-true-positive-true-negative-false-positive-and-fal
        FP = CM[0,1]
        FN = CM[1,0]
        TP = CM[1,1]
        TN = CM[0,0]
        acc = (TP+TN)/(TP+FP+FN+TN)
        TPR = TP/(TP+FN)
        TNR = TN/(TN+FP)
        
        #Report out
        TPR_scores[i] = TPR
        TNR_scores[i] = TNR
        acc_scores[i] = acc
        
        #/for loop

    #Visualize results
    plt.figure(figsize=(12,6))
    plt.plot(depths, acc_scores, label = 'accuracy')
    plt.plot(depths, TPR_scores, label = 'true positive rate')
    plt.plot(depths, TNR_scores, label = 'true negative rate')

    plt.xlabel('depth of tree')
    plt.title('Decision Tree: Model Performance with Varying Depths')
    plt.xticks(np.arange(0,22,2))
    plt.legend(loc="best")

We now want to examine the best tree to see what it’s doing:

    #First, make the best tree.
    my_depth = 2
    best_tree = DecisionTreeClassifier(max_depth=my_depth, random_state=2022).fit(X_train, Y_train)
    y_pred = best_tree.predict(X_train)
    y_eval_pred = best_tree.predict(X_eval)

    #Print out its scores:

    #Training
    CM = confusion_matrix(Y_train, y_pred) #Do NOT want to normalize this one—see below.
    #Thanks, lucidv01d! https://stackoverflow.com/questions/31324218/scikit-learn-how-to-obtain-true-positive-true-negative-false-positive-and-fal
    FP = CM[0,1]
    FN = CM[1,0]
    TP = CM[1,1]
    TN = CM[0,0]
    acc = (TP+TN)/(TP+FP+FN+TN)
    TPR = TP/(TP+FN)
    TNR = TN/(TN+FP)

    print('Training accuracy: ', acc)
    print('Training TPR: ', TPR)
    print('Training TNR: ', TNR, '\n\n\n')

    #Validation
    CM = confusion_matrix(Y_eval, y_eval_pred) #Do NOT want to normalize this one—see below.
    #Thanks, lucidv01d! https://stackoverflow.com/questions/31324218/scikit-learn-how-to-obtain-true-positive-true-negative-false-positive-and-fal
    FP = CM[0,1]
    FN = CM[1,0]
    TP = CM[1,1]
    TN = CM[0,0]
    acc = (TP+TN)/(TP+FP+FN+TN)
    TPR = TP/(TP+FN)
    TNR = TN/(TN+FP)

    print('Validation accuracy: ', acc) 
    print('Validation TPR: ', TPR)
    print('Validation TNR: ', TNR, '\n\n\n')

    #See the feature importances:
    print('Decision Tree Feature Importances:')
    for i , var in enumerate(variables):
      print('{0:5s}  {1:>-10.4f}'.format(var, np.round(best_tree.feature_importances_[i],4)))  

    #Since the depth is small, it should be pretty easy to see what the tree is actually doing:
    from sklearn import tree
    fig = plt.figure(figsize=(30,16))
    _ = tree.plot_tree(best_tree, feature_names=variables, class_names = ['no fire', 'fire'], filled=True, fontsize=12)
    plt.show()

Random Forest

Can we improve on the single decision tree by using ensemble methods? To
find out, first I’ll try random forest and then I’ll try adaptive
boosting.

    #Using the same reasoning as for the single decision tree above, I decided to use the real-units data for the random forest.
    #I'm not using the OOB scores because I want to treat false positives and negatives separately, as I've been doing.

    from sklearn.ensemble import RandomForestClassifier

    n_estimators = np.concatenate((np.arange(10, 100, 10), np.arange(100, 501, 100)))
    acc_scores_train = np.zeros(len(n_estimators))
    acc_scores_eval = np.zeros(len(n_estimators))

    TPR_scores_train = np.zeros(len(n_estimators))
    TPR_scores_eval = np.zeros(len(n_estimators))

    TNR_scores_train = np.zeros(len(n_estimators))        
    TNR_scores_eval = np.zeros(len(n_estimators))        

    for i, n_est in enumerate(n_estimators):
        rf = RandomForestClassifier(n_estimators=n_est, max_features="sqrt", oob_score=False, random_state=2022)
        rf.fit(X_train, Y_train)
        
        rf_train_pred = rf.predict(X_train)                
        rf_eval_pred = rf.predict(X_eval)
                                  
        CM_train = confusion_matrix(Y_train, rf_train_pred)
        FP = CM[0,1]
        FN = CM[1,0]
        TP = CM[1,1]
        TN = CM[0,0]
        acc = (TP+TN)/(TP+FP+FN+TN)
        TPR = TP/(TP+FN)
        TNR = TN/(TN+FP)
        
        acc_scores_train[i] = acc                         
        TPR_scores_train[i] = TPR
        TNR_scores_train[i] = TNR
                                  
        CM_eval = confusion_matrix(Y_eval, rf_eval_pred)
        FP = CM_eval[0,1]
        FN = CM_eval[1,0]
        TP = CM_eval[1,1]
        TN = CM_eval[0,0]
        acc = (TP+TN)/(TP+FP+FN+TN)
        TPR = TP/(TP+FN)
        TNR = TN/(TN+FP)
        
        acc_scores_eval[i] = acc                         
        TPR_scores_eval[i] = TPR
        TNR_scores_eval[i] = TNR
                                  

    #And plot:
    fig, ax = plt.subplots(2,1, sharex=True, figsize=(18,8))

    #First figure: training performance
    ax[0].set_title('Training Performance')                              
    ax[0].plot(n_estimators, TPR_scores_train, color = 'g', label = 'true positive rate')
    ax[0].plot(n_estimators, TNR_scores_train, color = 'b', label = 'true negative rate')
    ax[0].plot(n_estimators, acc_scores_train, color = 'k', label = 'accuracy')  
    ax[0].legend(loc = 'best')                              
                                  
    #Second figure: validation performance                              
    ax[1].set_title('Validation Performance')                              
    ax[1].plot(n_estimators, TPR_scores_eval, color = 'g', label = 'true positive rate')
    ax[1].plot(n_estimators, TNR_scores_eval, color = 'b', label = 'true negative rate')
    ax[1].plot(n_estimators, acc_scores_eval, color = 'k', label = 'accuracy')  
    ax[1].legend(loc = 'best')                                 

C Prepare the training and validation data

    #Get the training data formatted:

    #Clear away the observations with missing data
    culled_inputs, culled_labels = cull(inputs, labels)

    #New features
    avg_nbr_feat = avg_neighbor_batch(culled_inputs[:, :, :, 11], city_radius = 2) #so: 5x5 grid ctrd at each pixel is considered
    firewind_scores_feat = calc_firewind_scores(culled_inputs)

    #Vectorize input data for training
    #Supplied features
    flat_elevation = vectorize(culled_inputs[:,:,:,0])
    flat_wind_dir = vectorize(culled_inputs[:,:,:,1])
    flat_wind_speed = vectorize(culled_inputs[:,:,:,2])
    flat_min_temp = vectorize(culled_inputs[:,:,:,3])
    flat_max_temp = vectorize(culled_inputs[:,:,:,4])
    flat_humidity = vectorize(culled_inputs[:,:,:,5])
    flat_precipitation = vectorize(culled_inputs[:,:,:,6])
    flat_drought_index = vectorize(culled_inputs[:,:,:,7])
    flat_vegetation = vectorize(culled_inputs[:,:,:,8])
    flat_population_density = vectorize(culled_inputs[:,:,:,9])
    flat_ERC = vectorize(culled_inputs[:,:,:,10])
    flat_prev_masks = vectorize(culled_inputs[:,:,:,11])

    #Derived features
    flat_avg_nbrs = vectorize(avg_nbr_feat)
    flat_firewind_scores = vectorize(firewind_scores_feat)

    #Vectorize output data for training
    flat_labels = vectorize(culled_labels)
    Y_train = flat_labels

    #Load the validation data


    side_length = 32 #length of the side of the square you select (so, e.g. pick 64 if you don't want any random cropping)
    num_obs = 100 #batch size

    dataset = get_dataset(
          file_pattern,
          data_size=64,
          sample_size=side_length,
          batch_size=num_obs,
          num_in_channels=12,
          compression_type=None,
          clip_and_normalize=False,
          clip_and_rescale=False,
          random_crop=True,
          center_crop=False)

    inputs_eval, labels_eval = next(iter(dataset)) 

    #Toss out the observations with missing data
    culled_inputs_eval, culled_labels_eval = cull(inputs_eval, labels_eval)

    #New features
    avg_nbr_feat_eval = avg_neighbor_batch(culled_inputs_eval[:, :, :, 11], city_radius = 2) #so: 5x5 grid ctrd at each pixel is considered
    firewind_scores_feat_eval = calc_firewind_scores(culled_inputs_eval)

    #Vectorize input data for training
    #Supplied features
    flat_elevation_eval = vectorize(culled_inputs_eval[:,:,:,0])
    flat_wind_dir_eval = vectorize(culled_inputs_eval[:,:,:,1])
    flat_wind_speed_eval = vectorize(culled_inputs_eval[:,:,:,2])
    flat_min_temp_eval = vectorize(culled_inputs_eval[:,:,:,3])
    flat_max_temp_eval = vectorize(culled_inputs_eval[:,:,:,4])
    flat_humidity_eval = vectorize(culled_inputs_eval[:,:,:,5])
    flat_precipitation_eval = vectorize(culled_inputs_eval[:,:,:,6])
    flat_drought_index_eval = vectorize(culled_inputs_eval[:,:,:,7])
    flat_vegetation_eval = vectorize(culled_inputs_eval[:,:,:,8])
    flat_population_density_eval = vectorize(culled_inputs_eval[:,:,:,9])
    flat_ERC_eval = vectorize(culled_inputs_eval[:,:,:,10])
    flat_prev_masks_eval = vectorize(culled_inputs_eval[:,:,:,11])

    #Derived features
    flat_avg_nbrs_eval = vectorize(avg_nbr_feat_eval)
    flat_firewind_scores_eval = vectorize(firewind_scores_feat_eval)

    #Vectorize output data for evaluation data
    flat_labels_eval = vectorize(culled_labels_eval)
    Y_eval = flat_labels_eval


    y_pred = []
    threshold = 0.1 #set later, maybe with help of ROC curve or similar; for now just choose it.
    for i in range(len(y_pred_raw)):
        if y_pred_raw[i] >= threshold:
            y_pred.append(1)   
        else:                                                          
            y_pred.append(0)
    y_pred = np.array(y_pred)

    CM = confusion_matrix(flat_labels, y_pred, normalize = 'all') #since I want to calculate

    f, ax = plt.subplots(figsize=(9,9))
    disp = ConfusionMatrixDisplay(CM, display_labels=[0, 1]);
    disp.plot(ax=ax)
    ax.set_title('Confusion Matrix: Linear Classifier (Training Data)')
    plt.show()
    '''